# Traitement des données pour un fichier

Le but de ce fichier est de présenter le pré-traitement des données, avant l'utilisation d'un module d'IA.<br>
Pour réliser cela il faut :
- réduire au maximum le nombre de points pour permettre un apprentissage efficace même sur peu de données
- conserver toutes les informations permettant de différencier une courbe normale d'une courbe de malade


Import des modules utiles :
- `pandas` pour le traitement des données 
- `numpy` pour les calcules sur les vecteurs
- `scipy.signal` pour faire du traitement du signal

Remarque : on utilisera `plotly` pour l'affichage des courbes, il faut donc l'avoir installé.

In [41]:
import pandas as pd
import numpy as np
from scipy.signal import butter, lfilter

pd.options.plotting.backend = "plotly"

Définition des constantes qui seront utiles dans la suite du programme :
- `PATH`, le chemin jusqu'au fichier csv étudié
- `PERIOD`, la periode de ré-échantillonage des données en milliseconde, permettant de conserver plus ou moins de points sur la courbe
- `INTERVAL`, l'intervalle de temps contenant les informations significatives en minutes
- `NOISE_FREQ`, fréquence de coupure du filtre passe-bas permettant d'éliminer le bruit en Hz, valeur de None si il n'y a pas de filtrage souhaité
- `BIAS_FREQ`, fréquence de coupure du filtre passe-bas permettant de déterminer le biais sur la courbe en Hz
- `THRESHOLD`, seuil à partir duquel on considère une valeur comme significative dans la mesure.
- `SPIKES`, temps d'apparition en minute des trois pics de référence, valeur de None si le pic n'apparait pas

In [42]:
PATH = '../donnees/export-chromato/AbaziM-20200210-CHROMATOGRAM.CSV'
PERIOD = 5000 
RESAMPLE_MS = str(PERIOD) +'ms'
INTERVAL = [6,45]
NOISE_FREQ = 0.09
BIAS_FREQ = 0.01 
THRESHOLD = 1.7
SPIKES = [21.5,25.6, 30.2] 

## Lecture d'un fichier csv et affichage des données
On utilise pandas pour lire directement le fichier csv et mettre les données dans un objet du type DataFrame.<br>
Les 5 premières lignes ne sont pas prises en compte car elles correspondent aux métadonnées de l'analyse. <br>
Le parser détecte trois colonnes car les lignes finissent par une virgule, il faut donc supprimer la troisième colonne qui est vide du DataFrame.

In [43]:
# lecture du fichier csv
df = pd.read_csv(PATH, header=None, skiprows=[0,1,2,3,4,5], index_col=0)
#suppression de la colonne vide
df.drop(df.columns[1], axis=1, inplace=True)
df.rename(columns = {df.columns[0]: 'values'}, inplace=True)

On ne garde que les valeurs comprises dans l'intervalle qui nous interesse, cela supprime aussi les cases éventuellement vide du csv (temps de 0 à la lecture)<br>
Affichage des données brutes

In [44]:
df = df.drop(df[df.index > INTERVAL[1]].index)
df = df.drop(df[df.index < INTERVAL[0]].index)
df.plot()

## Mise en évidence des pics
Le passage des données en logarithme permet de mettre en évidence les plus petits pics du graphique, le but est de ne pas donner un importance trop grande aux pics les plus grands, car cela pourrait compliquer le travail sur la partie IA dans le cas ou l'information importante est contenue dans un pic de taille faible.<br>
Remarque : la fonction logarithme est bijective, il n'y a donc pas de perte d'information.

In [45]:
df = np.log(df)
df.plot()

Ensuite on fait une normalisation des points de la courbe

In [46]:
df = (df - df.mean())/df.std()
df.plot()

## Intervalles de mesures constant
La machine faisant les mesures n'a pas une période d'échantillonage constante, cela est problématique pour l'exploitation des données car chaque courbes aura des points d'échantillonage différents. <br>
On utilise donc la fonction `resample` de `pandas` qui permet de donner un échantilonnage constant Associé à `max` qui permet de conserver le maximum des points qui se trouveront entre deux points d'échantilonnage, dans le but de conserver la taille des différents pics.<br>
Ensuite `interpolate` permet de faire une interpolation polynomiale permettant de completer les points qui n'ont pas encore de valeur.

Avant de faire ceci il faut commencer par convertir le temps qui est un valeur en minutes dans un format compréhensible par `pandas`.

In [47]:
df['time'] = pd.to_timedelta(df.index, 'min')  # conversion de la durée en timedelta pour pouvoir faire un resample
df.set_index('time', inplace=True) # met le temps en indice du DataFrame

newdf = df.resample(rule=RESAMPLE_MS).max().interpolate(method='polynomial', order=3) # on change l'échantillonage pour qu'il soit constant.
newdf.rename(columns = {df.columns[0]: 'resampled'}, inplace=True)
print('nouveau nombre de valeur : ', len(newdf), '  ancien nombre de valeur : ', len(df))
newdf.plot()

nouveau nombre de valeur :  468   ancien nombre de valeur :  6608


## Filtrage et suppression du biais
On remarque sur la courbe ci-dessus que la machine à tendance à ajouter un biais au cours du temps. Les valeurs en 0 sont proche de -1 alors qu'en 45 elle sont plutot proche de 0.4. <br>
L'idée est donc de récupérer la composante basse fréquence de la courbe correspondant à ce biais par un filtre passe-bas, et ensuite de la soustraire aux valeurs de la courbe pour le comprenser. <br>
Une second filtre passe-bas pourra être utilisé pour supprimer les hautes fréquences, éliminant le bruit de mesure.

Mise en place du filtre passe-bas avec `butter` de `scipy`.<br>
Tiré de : <https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units>

In [48]:
# création du filtre
def butter_lowpass(cutoff, ws, order=5):
    nyq = 0.5 * ws
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

# application du filtre
def butter_lowpass_filter(data, cutoff, fs, order=5):
    cutoff *= 2*np.pi # conversion en rad/s
    ws  = fs * 2 * np.pi
    b, a = butter_lowpass(cutoff, ws, order=order)
    y = lfilter(b, a, data)
    return y

newdf['bias'] = butter_lowpass_filter(newdf, BIAS_FREQ, 1/PERIOD*1000)
newdf.plot()

Filtrage des hautes fréquences et suppression du biais :

In [49]:
if (NOISE_FREQ is None):
    newdf['corrected'] = newdf['resampled'] - newdf['bias']
else :
    newdf['corrected'] = butter_lowpass_filter(newdf['resampled'], NOISE_FREQ, 1/PERIOD*1000) # on perd la hauteur relative entre les pics parfois
    newdf['corrected'] = newdf['corrected'] - newdf['bias']
newdf.plot()

Pour éliminer les données parasites restantes, l'idée est de faire un seuillage pour ne conserver que les valeurs significatives sur la courbe et ainsi éliminer l'impact du bruit de mesure.

In [50]:
newdf.loc[newdf['corrected'] < THRESHOLD, 'corrected'] = 0
print('Nombre de valeurs restantes : ', len(newdf['corrected']))
newdf['corrected'].plot()

Nombre de valeurs restantes :  468
